In [3]:
import torch
import torch.nn as nn

In [5]:
class Ensemble(nn.Module):
    def __init__(self,model1,model2,model3, num_classes):
        super(Ensemble,self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3

    def forward(self,inputs):
        a = self.model1(inputs)
        b = self.model2(inputs)
        c = self.model3(inputs)
        x = a+b+c
        return x 


In [ ]:
model1 = myVgg(num_classes=10)
model2 = vgg11(num_classes=10)
model3 = alrex(num_classes=10)

ensem_model = Ensemble(model1,model2,model3,num_classes=10)
ensem_model.to(device)

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if device =='cuda':
            images = images.to(device)
            labels = labels.to(device)
        
        output = ensem_model(images)
        _, predicted = torch.max(output,1)
        